# Übungsblatt 6

**Lernziele**

In den Übungen dieser Wochen lernen Sie:

* Funktionen der Standardbibliothek zum Umgang mit Listen
* Records
* Aufzählungstypen

## Aufgabe 6.1 (P) Java und OCaml: Generics und Polymorphie, Wert- und Referenzgleichheit, Namespacing

Ähnlich wie wir mit Funktionen über Werte abstrahieren können, geht dies auch für Typen.
Es lassen sich Funktionen definieren, die auf beliebigen Typen funktionieren, genauso wie sich generische Typen mit Typvariablen definieren lassen.
In Java heißt dieses Feature Generics, im Kontext von funktionalen Sprachen spricht man von Polymorphie und abstrakten Datentypen.

Geben Sie eine entsprechende OCaml-Implementierung für den folgenden Java-Code.

```java
class Pair<L,R> {
    L left;
    R right;

    public Pair(L left, R right) {
        this.left = left;
        this.right = right;
    }

    @Override
    public boolean equals(Object o) {
        if (!(o instanceof Pair)) return false;
        Pair pair = (Pair) o;
        return this.left.equals(pair.left) &&
            this.right.equals(pair.right);
    }

    static <L, R> Pair<R, L> flip(Pair<L, R> pair) {
        return new Pair<R, L>(pair.right, pair.left);
    }
}

public class Generics {
    public static void main(String[] args) {
        Pair<Integer, String> a = new Pair<>(1, "one");
        Pair<Integer, String> b = new Pair<>(1, "one");
        System.out.println(a == b); // Referenzgleichheit: false
        System.out.println(a.equals(b)); // Wertgleichheit: true
        System.out.println(Pair.flip(a).left); // "one"
    }
}
```

In [1]:
[]

- : 'a list = []


In [2]:
(1, "hallo")

- : int * string = (1, "hallo")


In [3]:
module Pair =
  struct
    let flip (a, b) = (b, a)
  end

module Pair : sig val flip : 'a * 'b -> 'b * 'a end


In [4]:
Pair.flip (1, "hallo")

- : string * int = ("hallo", 1)


## Aufgabe 6.2 (P) Exceptions keine Option?

Viele Funktionen der Standardbibliothek wie z.B. `List.find : ('a -> bool) -> 'a list -> 'a` werfen eine Exception (in diesem Fall z.B. `Not_found` bei `List.find ((<) 2) [1;2]`. Warum ist dies gefährlich?

In [5]:
List.find ((<) 2) [1;2]

error: error

Operatoren sind auch nur normale Funktionen, die neu definiert werden können.

In [6]:
let (+) = (-) in 3 + 2

- : int = 1


In unseren Implementierungen wollen wir deshalb stattdessen den Typ `'a option` benutzen. 
Im Interpreter (z.B. utop) können Sie sich Definitionen verschiedener Identifier ausgeben lassen.
So liefert uns `#show_type option;;`:

In [7]:
#show_type option

type nonrec 'a option = None | Some of 'a


*Bemerkung*: Wert-Definitionen sind standardmäßig nicht-rekursiv, Typ-Definitionen hingegen standardmäßig rekursiv (siehe `'a tree` weiter unten).
Das `nonrec` ist hier aber nicht relevant (rechts kein ungebundener Typ).

Im folgenden Beispiel wird gezeigt, wie der `option` Typ verwendet werden kann, um eine `find_opt` Funktion zu definieren, die keine Exceptions wirft.

In [8]:
let rec find_opt f = function
  [] -> None
| x :: xs -> if f x then Some x else find_opt f xs

val find_opt : ('a -> bool) -> 'a list -> 'a option = <fun>


Da es mühsam wäre jedes Mal ein Pattern-Matching auf Option-Werte zu machen,
definieren Sie eine Funktion `(|?) : 'a option -> 'a -> 'a` welche im Falle von `None` den rechten Wert liefert.

In [9]:
let (|?) a b = match a with None -> b | Some c -> c

val ( |? ) : 'a option -> 'a -> 'a = <fun>


Zum Beispiel soll folgender Ausdruck für `[1;2;3;4]` den Wert `2` und für `[1;3]` den Wert `0` liefern.

In [10]:
let f xs = find_opt (fun x -> x mod 2 = 0) xs |? 0

val f : int list -> int = <fun>


In [11]:
f [1;2;3;4]

- : int = 2


In [12]:
f [1;3]

- : int = 0


## Aufgabe 6.3 (P) Binärbaum

Gegeben sei folgende Typ-Definition eines binären Baumes:

In [13]:
type 'a tree = Node of 'a * 'a tree * 'a tree | Leaf

type 'a tree = Node of 'a * 'a tree * 'a tree | Leaf


Hier ein Beispielbaum zum Testen:

In [14]:
let tree = Node (5, Node (2, Node (1, Leaf, Leaf), Node (3, Leaf, Leaf)), Node (7, Leaf, Leaf))

val tree : int tree =
  Node (5, Node (2, Node (1, Leaf, Leaf), Node (3, Leaf, Leaf)),
   Node (7, Leaf, Leaf))


Implementieren Sie die Funktion

```ocaml
val insert : 'a -> 'a tree -> 'a tree
```

die einen Wert in den Baum einfügt und den neuen Baum zurückliefert. Beim Einfügen
in den Suchbaum sollen kleinere Elemente links im Baum zu finden sein.

In [15]:
let rec insert a t =
  match t with
    Leaf -> Node (a, Leaf, Leaf)
  | Node (x, l, r) ->
      if a < x then
        Node (x, insert a l, r)
      else if a > x then
        Node (x, l, insert a r)
      else
        Node (x, l, r)

val insert : 'a -> 'a tree -> 'a tree = <fun>


Implementieren Sie die Funktion

```ocaml
val height : 'a tree -> int
```

die die Höhe des Baumes berechnet. Ein Blatt hat dabei die Höhe 0.

In [16]:
let rec height t =
  match t with
    Leaf -> 0
  | Node (_, l, r) -> max (height l) (height r) + 1

val height : 'a tree -> int = <fun>


Implementieren Sie die Funktionen

```ocaml
val min_elem : 'a tree -> 'a option
```

und

```ocaml
val max_elem : 'a tree -> 'a option
```

die das kleinste bzw. größte Element eines Baumes zurückliefern, sofern ein solches existiert.

In [17]:
let rec min_elem t =
  match t with
    Leaf -> None
  | Node (x, l, r) ->
      if l = Leaf then
        Some x
      else
        min_elem l

val min_elem : 'a tree -> 'a option = <fun>


In [18]:
let rec max_elem t =
  match t with
    Leaf -> None
  | Node (x, l, r) ->
      if r = Leaf then
        Some x
      else
        max_elem r

val max_elem : 'a tree -> 'a option = <fun>


Implementieren Sie die Funktion

```ocaml
val remove : 'a -> 'a tree -> 'a tree
```

die ein Element aus dem Baum entfernt und den neuen Baum zurückliefert.

In [19]:
let rec remove a t =
  match t with
    Leaf -> Leaf
  | Node (x, l, r) ->
      if a < x then
        Node (x, remove a l, r)
      else if a > x then
        Node (x, l, remove a r)
      else match (max_elem l) with
        Some m -> Node (m, remove m l, r)
      | None -> r

val remove : 'a -> 'a tree -> 'a tree = <fun>
